In [1]:
import $ivy.`com.chuusai::shapeless:2.3.2`
import $ivy.`org.typelevel::cats-core:0.9.0`

import $ivy.$                             

import $ivy.$                               

In [2]:
import cats._, cats.implicits._
import shapeless.{Id => _, _}, shapeless.ops.function._

import cats._, cats.implicits._

import shapeless.{Id => _, _}, shapeless.ops.function._

In [3]:
implicit class HListApplyOps[F[_] : Apply, L <: HList](fl: F[L]) {
  def @:[A](fa: F[A]): F[A :: L] = fa.map(a => a :: (_: L)) ap fl
}

implicit class HNilApplyOps(h: HNil) {
  def @:[F[_] : Functor, A](fa: F[A]): F[A :: HNil] = fa.map(_ :: h)
}

class HListSyntax[A <: HList, F <: AnyRef](a:A) {
  def $$[U](f: F)(implicit cftp: FnToProduct.Aux[f.type, A => U]): U = cftp(f)(a)
}

class HListSyntaxM[M[_], A <: HList, F <: AnyRef](ma: M[A]) {
  def map$[U](f: F)(implicit cftp: FnToProduct.Aux[f.type, A => U], m: Functor[M]): M[U] = m.map(ma)(a => cftp(f)(a))
  def flatMap$[U, B](f: F)(implicit
                           cftp: FnToProduct.Aux[f.type, A => U],
                           ev2: U <:< M[B],
                           m: Monad[M]
                          ): M[B] = m.flatMap(ma)(t => cftp(f)(t))
}

class ProductSyntax[T, A <: HList, F <: AnyRef](t:T)(implicit ev: Generic.Aux[T, A]) {
  def $$[U](f: F)(implicit cftp: FnToProduct.Aux[f.type, A => U]): U = cftp(f)(ev.to(t))
}

class ProductSyntaxM[M[_], T, A <: HList, F <: AnyRef](ft: M[T])(implicit ev: Generic.Aux[T, A]) {
  def map$[U](f: F)(implicit cftp: FnToProduct.Aux[f.type, A => U], m: Functor[M]): M[U] = m.map(ft)(t => cftp(f)(ev.to(t)))
  def flatMap$[U, B](f: F)(implicit
                           cftp: FnToProduct.Aux[f.type, A => U],
                           ev2: U <:< M[B],
                           m: Monad[M]
                          ): M[B] = m.flatMap(ft)(t => cftp(f)(ev.to(t)))
}

implicit def mkSyntax[A <: HList, F <: AnyRef](a: A)(implicit ffp: FnFromProduct.Aux[A => Any, F]): HListSyntax[A, F] = new HListSyntax[A, F](a)

implicit def mkSyntax2[M[_], A <: HList, F <: AnyRef](a: M[A])(implicit ffp: FnFromProduct.Aux[A => Any, F]): HListSyntaxM[M, A, F] = new HListSyntaxM[M, A, F](a)

implicit def mkSyntax3[T, A <: HList, F <: AnyRef](t: T)(implicit ev: Generic.Aux[T, A], ev2: FnFromProduct.Aux[A => Any, F]): ProductSyntax[T, A, F] =
  new ProductSyntax[T, A, F](t)

implicit def mkSyntax4[M[_], T, A <: HList, F <: AnyRef](t: M[T])(implicit ev: Generic.Aux[T, A], ev2: FnFromProduct.Aux[A => Any, F]): ProductSyntaxM[M, T, A, F] =
  new ProductSyntaxM[M, T, A, F](t)

defined class HListApplyOps
defined class HNilApplyOps
defined class HListSyntax
defined class HListSyntaxM
defined class ProductSyntax
defined class ProductSyntaxM
defined function mkSyntax
defined function mkSyntax2
defined function mkSyntax3
defined function mkSyntax4

In [4]:
case class X(i: Int, j: Int)

defined class X

In [5]:
(1 :: 2 :: HNil) $$ (_ + _)     //  3
(1, 2) $$ (_ + _)               //  3
(1, 2, 3, 4) $$ (_ * _ * _ * _) // 24
X(1, 2) $$ (_ + _)              //  3

List((1, 2)) map$ (_ + _)      //List(3)
List(1::2::HNil) map$ (_ + _)  //List(3)

List((1, 2)) flatMap$ ((a,b) => List(a+b))      //List(3)
List(1::2::HNil) flatMap$ ((a,b) => List(a+b))  //List(3)

//the punchline...

List(1,2,3) @: List(1,2) @: HNil map$ (_+_)  //List(2, 3, 3, 4, 4, 5)

res4_0: Int = 3
res4_1: Int = 3
res4_2: Int = 24
res4_3: Int = 3
res4_4: List[Int] = List(3)
res4_5: List[Int] = List(3)
res4_6: List[Int] = List(3)
res4_7: List[Int] = List(3)
res4_8: List[Int] = List(2, 3, 3, 4, 4, 5)